In [1]:
import cv2
import numpy as np

In [2]:
import torch

In [3]:
def try_func(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(e)
        return None

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/equip/best.pt')
model.to(device)

Using cache found in C:\Users\kimwo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-24 Python-3.10.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [5]:
colors = [
    (255, 255, 255),
    (255, 0, 0),
    (255, 0, 255),
    (255, 0, 0),
    (255, 255, 0),
    (255, 0, 0),
]

In [6]:
cap = cv2.VideoCapture(0)
def test():
    fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
    delay = int(1000/fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        results = model(frame, size=640)

        result = frame.copy()
        for i in range(len(results.xyxy[0])):
            r = results.pandas().xyxy[0].iloc[i, :].values.tolist()
            
            x1, y1, x2, y2, confidence, cls_id, cls_name = r
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            c = colors[cls_id]
            name = f"{cls_id}. {cls_name} : {confidence:.2f}"

            cv2.rectangle(result, (x1, y1), (x2, y2), c, 2)
            cv2.putText(result, name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2)


        cv2.imshow('frame',frame)
        cv2.imshow('box 0.5', np.squeeze(results.render(0.5)))
        cv2.imshow('box 0.75', np.squeeze(results.render(0.75)))
        cv2.imshow('result', result)

        if cv2.waitKey(1) & 0xff == 27:
            break
try:
    test()
except Exception as e:
    raise e
finally:
    try_func(cap.release)
    try_func(cv2.destroyAllWindows)


print("Goodbye User")

Goodbye User


In [7]:
try_func(cap.release)
try_func(cv2.destroyAllWindows)